In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
from torchvision.models.vgg import VGG

In [2]:
from torch.optim import lr_scheduler                                                                                                                                                                                       
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
import numpy as np
import time
import os
from PIL import Image
import torchvision.transforms as transform

In [3]:
class VGG16_model(nn.Module):
    def __init__(self, numClasses=6):
        super(VGG16_model, self).__init__()
        self.vgg16 = nn.Sequential(
            nn.Conv2d(1, 64, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, numClasses)
        )

    def forward(self, x):
        x = self.vgg16(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [4]:
batch_size = 4
epochs     = 200  #500
lr         = 1e-4
momentum   = 0
w_decay    = 1e-5
step_size  = 50
gamma      = 0.5
model_use  = "aoi_model" 

n_class = 6

In [5]:
import pandas as pd
data = pd.read_csv("./aoi/train.csv")

In [6]:
FullPath = os.getcwd()
data_dir  = os.path.join(FullPath + "/aoi/train_images")
model_dir = os.path.join(FullPath + "/models", model_use)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [7]:
composed = transform.Compose([transform.Resize((224,224)), #  resize
                               #transform.RandomCrop(300), # random crop
                               transform.ToTensor(),
                               transform.Normalize(mean=[0.5],  # normalize
                                                    std=[0.5])])

In [8]:
data_images = []
for i in range(data.shape[0]):
    images_path = (os.path.join(data_dir + "/" +data.ID[i]))
    images = Image.open(images_path)
    images = composed(images)
    data_images.append(np.array(images))

In [9]:
def split_data(data):
    tra = []
    #val = []
    for i, d in enumerate(data):
        #if i <= 2022:
        tra.append(d)
        #else:
        #    val.append(d)
    return tra

In [10]:
tra_data = zip(data_images, np.array(data.Label))
tra = split_data(tra_data)

In [11]:
tra_loader = DataLoader(tra,shuffle=True,batch_size = 4, num_workers = 0)
#val_loader = DataLoader(val,num_workers = 0,batch_size = 3)

In [12]:
use_gpu = torch.cuda.is_available()
num_gpu = list(range(torch.cuda.device_count()))

model = VGG16_model()

if use_gpu:
    ts = time.time()
    model = model.cuda()
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))
else:
#     nn.DataParallel(fcn_model)
    print("Use CPU to train.")

RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 7.92 GiB total capacity; 449.02 MiB already allocated; 21.19 MiB free; 4.98 MiB cached)

In [ ]:
print(model)
params = list(model.parameters())

In [ ]:
# define loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr = lr, momentum = momentum, weight_decay = w_decay)
# decay LR by a factor of 0.5 every step_size = 50 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)

In [ ]:
def train():
    for epoch in range(epochs):
        model.train()
        scheduler.step()
        
        configs    = "vgg16_{}_batch{}_epoch{}_RMSprop_lr{}"\
            .format(model_use, batch_size, epoch, lr)
        model_path = os.path.join(model_dir, configs)
        
        ts = time.time()
        for iter, (data, label) in enumerate(tra_loader):
            optimizer.zero_grad()
            #data = data.unsqueeze(1)
            if use_gpu:
                inputs = Variable(data.cuda())
                labels = Variable(label.cuda())
            else:
                inputs, labels = Variable(data), Variable(label)
            inputs = inputs.float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 1 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch+1, iter, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch+1, time.time() - ts))
        if epoch % 1 == 0:
            torch.save(model.state_dict(),model_path + '.pkl')

        #val(epoch)

In [ ]:
#train()

In [ ]:
model = VGG16_model()
test = pd.read_csv("./aoi/test.csv")
state_dict = torch.load(os.path.join(model_dir, "vgg16_aoi_model_batch4_epoch149_RMSprop_lr0.0001.pkl"), map_location='cpu')
model.load_state_dict(state_dict)
 
test_dir  = os.path.join(FullPath + "/aoi/test_images")
test_images = []
    
for i in range(test.shape[0]):
    images_path = (os.path.join(test_dir + "/" +test.ID[i]))
    images = Image.open(images_path)
    images = composed(images)
    test_images.append(np.array(images))
    
if use_gpu:
    ts = time.time()
    model = model.cuda()
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))
else:
#     nn.DataParallel(fcn_model)
    print("Use CPU to train.")

In [ ]:
test_loader = DataLoader(test_images,shuffle=False,batch_size = 4, num_workers = 0)

In [ ]:
result = []
for i,data in enumerate(test_loader):
    if use_gpu:
        inputs = Variable(data.cuda())
    else:
        inputs, labels = Variable(data)
    outputs = model(inputs)
    outputs = outputs.data.cpu().numpy()
    result.append(np.argmax(outputs, axis=1))

In [ ]:
result[2535]

In [ ]:
label = []
for i in range(2535):
    label.append(result[i][0])
    label.append(result[i][1])
    label.append(result[i][2])
    label.append(result[i][3])
label.append(result[2535][0])
label.append(result[2535][1])

In [ ]:
test.Label = label

In [ ]:
test.to_csv(os.path.join(FullPath + "/aoi/Result4.csv",index=False))